#### Faiss
Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning.

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader=TextLoader("speech.txt")
documents=loader.load()
text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs=text_splitter.split_documents(documents)

In [3]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

In [5]:
embeddings=OllamaEmbeddings(model="nomic-embed-text")
db=FAISS.from_documents(docs,embeddings)
db

In [6]:
### querying 
query="How does the speaker describe the desired outcome of the war?"
docs=db.similarity_search(query)
docs[0].page_content


'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### As a Retriever
We can also convert the vectorstore into a Retriever class. This allows us to easily use it in other LangChain methods, which largely work with retrievers

In [7]:
retriever=db.as_retriever()
docs=retriever.invoke(query)
docs[0].page_content

'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

#### Similarity Search with score
There are some FAISS specific methods. One of them is similarity_search_with_score, which allows you to return not only the documents but also the distance score of the query to them. The returned distance score is L2 distance. Therefore, a lower score is better.

In [8]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score

[(Document(id='0c323aa9-9e80-4902-a6b6-5dd19aa3290c', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  np.float32(0.777352)),
 (Document(id='8b5a2b5c-bd7e-4b0b-b858-a98f029485d5', metadata={'source': 'spee

In [9]:
embedding_vector=embeddings.embed_query(query)
embedding_vector

[-0.025585193,
 -0.0005220519,
 -0.16198291,
 -0.026022287,
 0.10032356,
 0.06715196,
 -0.04215592,
 -0.006946124,
 0.019571947,
 -0.028642189,
 -0.011064039,
 0.04238279,
 0.036765188,
 0.06635059,
 0.09457365,
 -0.048498333,
 0.024810616,
 -0.051619846,
 -0.04579228,
 0.029540127,
 -0.04588634,
 -0.014859236,
 0.0007944474,
 0.017363442,
 0.08850759,
 0.02939616,
 -0.012536292,
 0.041635215,
 -0.044480447,
 -0.021120554,
 0.051377192,
 -0.014853029,
 -0.011812277,
 0.007641984,
 -0.09059418,
 -0.06951158,
 0.02394462,
 0.046546917,
 0.020861315,
 -0.06357061,
 0.006452988,
 -0.0140150525,
 0.0015980048,
 -0.07048785,
 0.060858037,
 -0.006191053,
 -0.0028239498,
 0.015149673,
 0.009376728,
 -0.029954834,
 -0.030148804,
 -0.010329114,
 0.013527904,
 -0.0011241985,
 0.0536124,
 0.012273914,
 -0.022402173,
 -0.03948551,
 0.012175953,
 0.014610206,
 0.026417492,
 0.013464307,
 -0.027661279,
 0.05046197,
 0.05759418,
 -0.048693664,
 -0.05142467,
 0.025579853,
 0.029784845,
 -0.074220285,
 

In [10]:
docs_score=db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(id='0c323aa9-9e80-4902-a6b6-5dd19aa3290c', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='8b5a2b5c-bd7e-4b0b-b858-a98f029485d5', metadata={'source': 'speech.txt'}, page_content='…\n

In [11]:
### Saving And Loading
db.save_local("faiss_index")

In [12]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)
docs=new_db.similarity_search(query)

In [13]:
docs

[Document(id='0c323aa9-9e80-4902-a6b6-5dd19aa3290c', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='8b5a2b5c-bd7e-4b0b-b858-a98f029485d5', metadata={'source': 'speech.txt'}, page_content='…\n